In [1]:
from basicsr.archs.basicvsr_arch import BasicVSR
import torch
import numpy as np
from glob import glob
import cv2
from torchviz import make_dot
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

C:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = BasicVSR(num_feat=64, num_block=30)
model.load_state_dict(torch.load("./BasicVSR_REDS4-543c8261.pth")['params'], strict=True)

<All keys matched successfully>

In [3]:
model.to('cuda')
_ = model.eval()

In [4]:
frames = glob("video/*.jpg")
frames = sorted(frames)

In [5]:
inputs = []
for frame in frames[:5]:
    img = cv2.imread(frame)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    size = img.shape[:2]
    img = cv2.resize(img, (size[1]//2, size[0]//2))
    inputs.append(img)

In [6]:
inputs = np.array(inputs).astype(np.float32)
inputs /= 255.0

In [7]:
inputs.shape

(5, 72, 128, 3)

In [8]:
inputs = np.transpose(inputs, (0, 3, 1, 2))

In [9]:
inputs = torch.tensor(inputs).unsqueeze(0).to("cuda")

In [10]:
inputs.shape

torch.Size([1, 5, 3, 72, 128])

In [11]:
summary(model, input_size=inputs.shape[1:])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 32, 3, 4]          12,576
              ReLU-2             [-1, 32, 3, 4]               0
            Conv2d-3             [-1, 64, 3, 4]         100,416
              ReLU-4             [-1, 64, 3, 4]               0
            Conv2d-5             [-1, 32, 3, 4]         100,384
              ReLU-6             [-1, 32, 3, 4]               0
            Conv2d-7             [-1, 16, 3, 4]          25,104
              ReLU-8             [-1, 16, 3, 4]               0
            Conv2d-9              [-1, 2, 3, 4]           1,570
      BasicModule-10              [-1, 2, 3, 4]               0
           Conv2d-11             [-1, 32, 6, 8]          12,576
             ReLU-12             [-1, 32, 6, 8]               0
           Conv2d-13             [-1, 64, 6, 8]         100,416
             ReLU-14             [-1, 6

In [13]:
# with torch.no_grad():
outputs = model(inputs[:,:10])

In [12]:
outputs = outputs.to('cpu')
model = model.cpu()
make_dot(outputs.mean(), params=dict(model.named_parameters()))

In [55]:
outputs = outputs.squeeze(0).cpu().numpy()

In [56]:
outputs = np.transpose(outputs, (0, 2, 3, 1))

In [57]:
# os.mkdir("out")
for i, out in enumerate(outputs):
    out = out * 255
    out = out.astype(np.uint8)
    out = cv2.cvtColor(out, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f"out/out_{i}.jpg", out)

In [13]:
# model_traced.cuda()
outputs = model_traced(inputs[:, 10:])

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 5.01 GiB already allocated; 0 bytes free; 5.38 GiB reserved in total by PyTorch)



In [54]:
import cv2 
import numpy as np
from PIL import Image
import torch
from torch.nn.functional import interpolate
from glob import glob

In [57]:
imgs = glob("video/*.jpg")
imgs = [Image.open(im) for im in imgs]

In [61]:
Image.open(imgs[0])

AttributeError: module 'PIL.Image' has no attribute 'read'

In [58]:
imgs = np.array(imgs).astype(np.float32)
imgs /= 255.0
imgs = np.transpose(imgs, (0, 2, 3, 1)) 

C:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The input object of type 'JpegImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'JpegImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  """Entry point for launching an IPython kernel.
C:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


TypeError: float() argument must be a string or a number, not 'JpegImageFile'

In [1]:
from basic_vsr import BasicVSR
from spynet import SpyNet
from ifnet import IFNet

import torch

c:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
spynet = SpyNet().to('cuda')
ifnet = IFNet().to('cuda')

basic_if = BasicVSR(ifnet).to('cuda')
basic_spy = BasicVSR(spynet).to('cuda')

In [3]:
x = torch.randn((1, 6, 256, 256)).to('cuda')

In [4]:
basic_if.get_flow(x)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
ifnet(x, flow_only=False)

RuntimeError: Given groups=1, weight of size [45, 11, 3, 3], expected input[1, 6, 64, 64] to have 11 channels, but got 6 channels instead